In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
from lightgbm import LGBMClassifier
from sklearn.metrics import auc,roc_curve,roc_auc_score,accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [ ]:

original = pd.read_csv('/kaggle/input/depression-surveydataset-for-analysis/final_depression_dataset_1.csv')
tr_data = pd.read_csv('/kaggle/input/playground-series-s4e11/train.csv')
te_data = pd.read_csv('/kaggle/input/playground-series-s4e11/test.csv')
df = pd.concat([original,tr_data,te_data])

In [ ]:
df.head(5)

In [ ]:
df.isnull().sum()

In [ ]:
df.shape

In [ ]:
df=df.drop(columns=['Profession','Job Satisfaction','Academic Pressure','CGPA','Study Satisfaction'],inplace=False)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
import matplotlib.pyplot as plt 
import seaborn as sns

In [ ]:
plt.hist(df['Work Pressure'], bins=10)#color='skyblue', edgecolor='black')

In [ ]:
mean_value=df['Work Pressure'].mean()
df['Work Pressure']=df['Work Pressure'].fillna(mean_value)

In [ ]:
df.isnull().sum()

In [ ]:

tr_data = pd.read_csv('/kaggle/input/playground-series-s4e11/train.csv')
te_data = pd.read_csv('/kaggle/input/playground-series-s4e11/test.csv')


In [ ]:
df.Depression=df.Depression.replace({'No':0,'Yes':1}).fillna(0).astype(int)

In [ ]:
df=pd.get_dummies(df,columns=df.select_dtypes(include='O').columns,drop_first=True)

In [ ]:
tr_data=df[:-te_data.shape[0]]
te_data=df[-te_data.shape[0]:]

In [ ]:
X=tr_data.drop(['id','Depression'],axis=1)
Y=tr_data.Depression

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=123,stratify=Y)

In [ ]:
classes = np.unique(y_train)
class_weights = compute_class_weight('balanced', classes=classes, y=y_train)
class_weights = dict(zip(classes, class_weights))

model = LGBMClassifier(learning_rate = 0.01,n_estimators = 1000,class_weight=class_weights)
model.fit(X_train,y_train)

In [ ]:
y_pred_prob = model.predict_proba(X_test)[:,1]
print('ROC-AUC Score:', roc_auc_score(y_test,y_pred_prob))

fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)
roc_auc = auc(fpr, tpr)

# Plot the ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='b', lw=2, label='ROC curve (AUC = {:.2f})'.format(roc_auc))
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')  

# Annotate major thresholds
for i in range(0, len(thresholds), int(len(thresholds) / 10)):
    plt.scatter(fpr[i], tpr[i], color='red', marker='o')
    plt.text(fpr[i] + 0.02, tpr[i] - 0.05, f'Accuracy:{accuracy_score(y_test,y_pred_prob>thresholds[i]):.5f}@{thresholds[i]:.2f}', color='red')
warnings.filterwarnings('ignore')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

In [ ]:
best_threshold = 0
accuracy = 0 
for i in thresholds:
    tmp = accuracy_score(y_test,y_pred_prob>i)
    if accuracy < tmp:
        best_threshold = i
        accuracy = tmp
print('Accuracy:',accuracy,'@ Threshold:',best_threshold)

In [ ]:
te_data['Depression'] = (model.predict_proba(te_data.drop(['id','Depression'],axis=1))[:,1]>best_threshold).astype(int)

In [ ]:
te_data[['id','Depression']].to_csv('submit.csv',index=False)